In [1]:
import tensorflow as tf
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 15,
    horizontal_flip= True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
mean = np.mean(x_train)
std = np.std(x_train)
x_test = (x_test - mean) / std
x_train = (x_train - mean) / std
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
def lr_schedule(epoch):
  if epoch > 200 :
    return 0.0001
  if epoch > 50 :
    return 0.001
  return 0.01
  
reduced_lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [3]:
def bottleneck(t, inc, ouc, n, s, x):
    x_input = x
    strides = [s] + [1] * (n - 1)
    for stride in strides:
        x = tf.keras.layers.Conv2D(filters = inc * t, kernel_size= (1, 1), strides = 1, padding = 'SAME'
                                   , kernel_initializer=tf.keras.initializers.he_normal())(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.nn.relu6(x)
        x = tf.keras.layers.DepthwiseConv2D(kernel_size= (3, 3), strides=stride,padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.nn.relu6(x)
        x = tf.keras.layers.Conv2D(filters = ouc, kernel_size=(1, 1), strides=1,  padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if stride == 1 :
            x_input = tf.keras.layers.Conv2D(filters = ouc, kernel_size=(1, 1), strides=1,  padding='same')(x)
            x = tf.keras.layers.Add()([x, x_input])
    return x
    

In [4]:
x_input = tf.keras.layers.Input(shape = (32,32,3))
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3),  strides= (1, 1), padding = 'SAME', kernel_initializer=tf.keras.initializers.he_normal())(x_input)
x = tf.keras.layers.BatchNormalization()(x)
x = bottleneck(1, 32, 16, 1, 1, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 16, 24, 2, 1, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 24, 32, 3, 1, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 32, 64, 4, 2, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 64, 96, 3, 1, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 96, 160, 3, 2, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = bottleneck(6, 160, 320, 1, 1, x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x = tf.keras.layers.Conv2D(filters = 1280, kernel_size=(1, 1), strides=1, padding='same', kernel_initializer=tf.keras.initializers.he_normal())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(units = 10, activation = 'softmax', kernel_initializer=tf.keras.initializers.he_normal())(x)
model = tf.keras.Model(inputs=x_input, outputs = x)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.045, momentum=0.9),
    metrics=['accuracy'],
)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 32)   1056        ['batch_normalization[0][0]']

c:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


                                                                                                  
 batch_normalization_49 (BatchN  (None, 8, 8, 960)   3840        ['conv2d_47[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.relu6_32 (TFOpLambda)    (None, 8, 8, 960)    0           ['batch_normalization_49[0][0]'] 
                                                                                                  
 depthwise_conv2d_16 (Depthwise  (None, 8, 8, 960)   9600        ['tf.nn.relu6_32[0][0]']         
 Conv2D)                                                                                          
                                                                                                  
 batch_normalization_50 (BatchN  (None, 8, 8, 960)   3840        ['depthwise_conv2d_16[0][0]']    
 ormalizat

In [5]:
model.fit(datagen.flow(x_train, y_train, batch_size=128), callbacks=[reduced_lr], steps_per_epoch = len(x_train) // 128, epochs = 250, validation_data=(x_test, y_test), verbose = 1)

Epoch 1/250
390/390 [==============================] - 149s 336ms/step - loss: 1.8455 - accuracy: 0.3108 - val_loss: 3.0893 - val_accuracy: 0.1000 - lr: 0.0100
Epoch 2/250
390/390 [==============================] - 148s 380ms/step - loss: 1.5075 - accuracy: 0.4483 - val_loss: 1.5728 - val_accuracy: 0.4433 - lr: 0.0100
Epoch 3/250
390/390 [==============================] - 398s 1s/step - loss: 1.3456 - accuracy: 0.5133 - val_loss: 1.4700 - val_accuracy: 0.5082 - lr: 0.0100
Epoch 4/250
390/390 [==============================] - 109s 280ms/step - loss: 1.2169 - accuracy: 0.5601 - val_loss: 1.4278 - val_accuracy: 0.5232 - lr: 0.0100
Epoch 5/250
390/390 [==============================] - 106s 271ms/step - loss: 1.1104 - accuracy: 0.6041 - val_loss: 1.0905 - val_accuracy: 0.6308 - lr: 0.0100
Epoch 6/250
390/390 [==============================] - 108s 278ms/step - loss: 1.0186 - accuracy: 0.6409 - val_loss: 1.1619 - val_accuracy: 0.6164 - lr: 0.0100
Epoch 7/250
390/390 [======================